In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from statsmodels.graphics import tsaplots
import statsmodels.api as sm
from statsmodels.tsa.arima_model import ARIMA, ARIMAResults, ARMA
from statsmodels.tsa.arima_process import ArmaProcess
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings("ignore")

/home/nicole/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
from timeseries_functions import index_to_datetime, plot_all_df_columns, plot_series,\
plot_series_save_fig, plot_series_and_differences, run_augmented_Dickey_Fuller_test, \
plot_autocorrelation, plot_partial_autocorrelation, plot_decomposition

### import data

In [3]:
duration_train_df = pd.read_csv('./data/hours_training_data.csv', parse_dates=True, index_col='AppointmentDate')

In [4]:
duration_test_df = pd.read_csv('./data/hours_test_data.csv', parse_dates=True, index_col='AppointmentDate')

In [5]:
all_data_df = pd.concat([duration_train_df, duration_test_df], axis=0)

In [6]:
all_data_df.isnull().sum()

Specialty              0
AppointmentDate.1      0
AppointmentDuration    0
Hours_Spent            0
dtype: int64

In [7]:
all_data_df.shape

(58830, 4)

#### all data

In [8]:
all_doctors = all_data_df[all_data_df['Specialty'] == 'doctor']
all_therapists = all_data_df[all_data_df['Specialty'] == 'therapist']
all_RN_PA = all_data_df[all_data_df['Specialty'] == 'RN/PA']

In [9]:
# Group by hours per day
all_doc_duration = all_doctors.groupby(all_doctors.index.date)['Hours_Spent'].sum()
all_RN_PA_duration = all_RN_PA.groupby(all_RN_PA.index.date)['Hours_Spent'].sum()
all_therapist_duration = all_therapists.groupby(all_therapists.index.date)['Hours_Spent'].sum()

In [10]:
all_data = [all_doc_duration, all_RN_PA_duration, all_therapist_duration]

In [11]:
for item in all_data:
    index_to_datetime(item)

In [12]:
all_doc_duration.head()

2015-01-05     4.50
2015-01-06    20.00
2015-01-07    17.00
2015-01-08    23.75
2015-01-09    12.00
Name: Hours_Spent, dtype: float64

In [13]:
all_dr = weekly_resample(all_doc_duration)
all_RN_PA = weekly_resample(all_RN_PA_duration)
all_therapist = weekly_resample(all_therapist_duration)

NameError: name 'weekly_resample' is not defined

In [ ]:
all_dr.head()

In [ ]:
# drop partial week, first row
all_dr = all_dr[1:]
all_RN_PA = all_RN_PA[1:]
all_therapist = all_therapist[1:]

In [ ]:
all_data_sets = [all_dr, all_RN_PA, all_therapist]

In [ ]:
all_dr.index
# all_RN_PA.index
# all_therapist.index

### export weekly hours data to csv for each category

In [ ]:
all_dr.to_csv('./data/all_dr_hours.csv')

In [ ]:
all_RN_PA.to_csv('./data/all_RN_PA_hours.csv')

In [ ]:
all_therapist.to_csv('./data/all_therapist_hours.csv')

### Separate training and test df by specialty, then downsample to get hours/week

#### training data

In [ ]:
doctors = duration_train_df[duration_train_df['Specialty'] == 'doctor']
therapists = duration_train_df[duration_train_df['Specialty'] == 'therapist']
RN_PA = duration_train_df[duration_train_df['Specialty'] == 'RN/PA']

In [ ]:
# Group by hours per day
doc_duration = doctors.groupby(doctors.index.date)['Hours_Spent'].sum()
RN_PA_duration = RN_PA.groupby(RN_PA.index.date)['Hours_Spent'].sum()
therapist_duration = therapists.groupby(therapists.index.date)['Hours_Spent'].sum()

In [ ]:
training_data = [doc_duration, RN_PA_duration, therapist_duration]

In [ ]:
# convert training data index to datetime
for item in training_data:
    index_to_datetime(item)

In [ ]:
training_data = [doc_duration, RN_PA_duration, therapist_duration]
for data in training_data:
    weekly_resample(data)

In [ ]:
dr_train_data = weekly_resample(doc_duration)
RN_PA_train_data = weekly_resample(RN_PA_duration)
therapist_train_data = weekly_resample(therapist_duration)

In [ ]:
# drop partial week, first row
dr_train_data = dr_train_data[1:]
RN_PA_train_data = RN_PA_train_data[1:]
therapist_train_data = therapist_train_data[1:]

In [ ]:
train_data = [dr_train_data, RN_PA_train_data, therapist_train_data]

#### test data

In [ ]:
dr_test_data = duration_test_df[duration_test_df['Specialty'] == 'doctor']
RN_PA_test_data = duration_test_df[duration_test_df['Specialty'] == 'RN/PA']
therapist_test_data = duration_test_df[duration_test_df['Specialty'] == 'therapist']

In [ ]:
dr_test_data = dr_test_data.groupby(dr_test_data.index.date)['Hours_Spent'].sum()
RN_PA_test_data = RN_PA_test_data.groupby(RN_PA_test_data.index.date)['Hours_Spent'].sum()
therapist_test_data = therapist_test_data.groupby(therapist_test_data.index.date)['Hours_Spent'].sum()

In [ ]:
test_data = [dr_test_data, RN_PA_test_data, therapist_test_data]

In [ ]:
# convert test data index to datetime
for item in test_data:
    index_to_datetime(item)

In [ ]:
dr_test_set = weekly_resample(dr_test_data)
RN_PA_test_set = weekly_resample(RN_PA_test_data)
therapist_test_set = weekly_resample(therapist_test_data)

In [ ]:
plot_series(series=dr_train_data, xlabel='', ylabel='', plot_name='Doctors')

### ARIMA models: weekly hours data

In [ ]:
def get_ARIMA_Model(data, order):
    "Fits ARIMA model"
    arima = ARIMA(data, order=order)
    results = arima.fit()
    summary = results.summary()
    params = results.params
    residuals = results.resid
    return results, summary, params, residuals

In [ ]:
# get doctors ARIMA and print summary
dr_results, dr_summary,dr_params, dr_residuals = get_ARIMA_Model(data=dr_train_data, order=(4,1,2))

In [ ]:
dr_summary

In [ ]:
# get RN/PA ARIMA and print summary
RN_PA_results, RN_PA_summary, RN_PA_params, RN_PA_residuals = get_ARIMA_Model(data=RN_PA_train_data, order=(4,1,2))

In [ ]:
RN_PA_summary

In [ ]:
# get therapist ARIMA and print summary
ther_results, ther_summary, ther_params, ther_residuals = get_ARIMA_Model(data=therapist_train_data, order=(4,1,1))

In [ ]:
ther_summary

In [ ]:
def plot_ARIMA_model(data, order, start, end, title='', xlabel='', ylabel=''):
    "Plots ARIMA model"
    results, summary, params, residuals = get_ARIMA_Model(data, order)
    fig = results.plot_predict(start=start, end=end)
    plt.title(title)
    plt.ylabel(xlabel)
    plt.xlabel(ylabel)
    plt.show()

In [ ]:
def plot_ARIMA_resids(data, order, start, end, title='', xlabel='', ylabel=''):
    "Plots ARIMA model residuals"
    results, summary, params, residuals = get_ARIMA_Model(data, order)
    residuals.plot(figsize=(5,5))
    plt.title(title)
    plt.ylabel(xlabel)
    plt.xlabel(ylabel)
    plt.show()

#### doctors

In [ ]:
params = {'figure.figsize': [12,8],'axes.grid.axis': 'both', 'axes.grid': True, 'axes.labelsize': 'Medium', 'font.size': 12.0, \
'lines.linewidth': 2}
plt.rcParams.update(params)
start= '2015-01-19'
end = '2018-07-30'
plot_ARIMA_model(data=dr_train_data, order=(4,1,2), start=start, end=end, title='Doctors ARIMA', xlabel='', ylabel='')

In [ ]:
plot_ARIMA_resids(data=dr_train_data, order=(4,1,2), start=3, end=20, title='Doctors Residuals', xlabel='', ylabel='')

#### RN/PAs

In [ ]:
params = {'figure.figsize': [12,8],'axes.grid.axis': 'both', 'axes.grid': True, 'axes.labelsize': 'Medium', 'font.size': 12.0, \
'lines.linewidth': 2}
plt.rcParams.update(params)
start= '2015-01-19'
end = '2018-07-30'
plot_ARIMA_model(data=RN_PA_train_data, order=(4,1,2), start=start, end=end, title='RN/PA ARIMA', xlabel='', ylabel='')

In [ ]:
plot_ARIMA_resids(data=RN_PA_train_data, order=(4,1,2), start=3, end=20, title='RN/PA Residuals', xlabel='', ylabel='')

#### therapists

In [ ]:
params = {'figure.figsize': [12,8],'axes.grid.axis': 'both', 'axes.grid': True, 'axes.labelsize': 'Medium', 'font.size': 12.0, \
'lines.linewidth': 2}
plt.rcParams.update(params)
start= '2015-01-19'
end = '2018-07-30'
plot_ARIMA_model(data=therapist_train_data, order=(4,1,1), start=start, end=end, title='Therapists ARIMA', xlabel='', ylabel='')

In [ ]:
plot_ARIMA_resids(data=therapist_train_data, order=(4,1,1), start=3, end=20, title='Therapists Residuals', xlabel='', ylabel='')

### ARIMA forcasts

In [ ]:
training_datasets = [dr_train_data, RN_PA_train_data, therapist_train_data]
test_sets = [dr_test_set, RN_PA_test_set, therapist_test_set]

In [ ]:
def get_ARIMA_forecast(data, order, start, end, typ=None):
    results = ARIMA(data, order=order).fit()
    forecast = results.predict(start=start, end=end, typ=typ)
    return forecast

In [ ]:
# doctors
get_ARIMA_forecast(data=dr_train_data, order=(4,1,2), start=3, end=20, typ='levels')

In [ ]:
# RN/PAs
get_ARIMA_forecast(data=RN_PA_train_data, order=(4,1,2), start=3, end=20, typ='levels')

In [ ]:
# therapists
get_ARIMA_forecast(data=therapist_train_data, order=(4,1,1), start=3, end=20, typ='levels')